In [ ]:
###################################################################################################
#
# resolution_comparison_plot.py 	(c) Ian Williams, Adnan Khan, Matt McQuinn
#     				    	        ianw89@live.com
#
#  This produces the plots in Appendix C of Khan, Williams, McQuinn 2022
#
###################################################################################################

import matplotlib.pyplot as plt
from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.ticker as mticker
import cgmbrush.plots.plots as plots

In [ ]:
date = '2022-06-03'
date2 = '2022-06-03'
provider = BolshoiProvider()
precipProfile = MassDependentProfile(PrecipitationProfile(), NFWProfile(), 10**14.1)

### PLOT SETTINGS ###
axis_fontsize = 12
curve_lw = 3
axis_label = 24
sub_title_size= 22

plt.rc('axes', titlesize=axis_fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=axis_label)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=XBIG_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=XBIG_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
plt.rc('figure', titlesize=sub_title_size)  # fontsize of the figure title

In [ ]:

STH1_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=1)
STH2_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=2)
STH4_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=4)
STH8_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=8)
STH16_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=16)
STH32_config = Configuration(SphericalTophatProfile(), provider=provider, resolution=32)

STH_configs = [STH1_config, STH2_config, STH4_config, STH8_config, STH16_config, STH32_config]
for c in STH_configs:
    c.datestamp = date

res = [1,2,4,8,16,32]
fire_configs = []
precipitation_configs = []
nfw_configs = []
for r in res:
    fire_configs.append(Configuration(FireProfile(), provider=provider, resolution=r))
    precipitation_configs.append(Configuration(precipProfile, provider=provider, resolution=r))
    nfw_configs.append(Configuration(NFWProfile(), provider=provider, resolution=r))
for c in fire_configs:
    c.datestamp = date2
for c in precipitation_configs:
    c.datestamp = date2
for c in nfw_configs:
    c.datestamp = date


In [ ]:

def make_resolution_image_comparison(vmax, c1, c2, c3, c4):
    """Image plots of methodology. Comparing resolutions of the halos re-added density field."""

    u=15
    vmin = 0

    fields = [
        ('Resolution: {}'.format(c1.resolution*1024), c1.get_addition_field()[0,0:c1.resolution*u,0:c1.resolution*u]),
        ('Resolution: {}'.format(c2.resolution*1024), c2.get_addition_field()[0,0:c2.resolution*u,0:c2.resolution*u]),
        ('Resolution: {}'.format(c3.resolution*1024), c3.get_addition_field()[0,0:c3.resolution*u,0:c3.resolution*u]),
        ('Resolution: {}'.format(c4.resolution*1024), c4.get_addition_field()[0,0:c4.resolution*u,0:c4.resolution*u])
    ]
    
    fig, axs = fields_comparison_plot(fields, vmin, vmax)
    saveFig('implot_resolution_comparison_%s.pdf' % vmax, fig, bbox_inches='tight')

 

def make_resolution_DM_histogram_comparison(data, filename='hist_resolution_comparison.pdf'):

    fig, axs = plt.subplots(1, len(data), sharex='col', sharey='row',
                            gridspec_kw={'hspace': 0, 'wspace': 0},figsize=(10*len(data),10))

    res_color_map = {
        '1024':'r',
        '2048':'g',
        '4096':'b',
        '8192':'k',
        '16384':'m',
        '32768':'c'
        }

    if len(data) == 1:
        axs = [axs]

    for i in range(len(data)):
        d = data[i]
        
        for c in d:
            hist = create_histograms(c.get_final_field(), c.resolution * c.provider.halofieldresolution)
            label = str(c.resolution * c.provider.halofieldresolution)

            axs[i].plot(hist[0], hist[1] / np.sum(hist[1]), '-', label=label, lw=curve_lw, color=res_color_map[label])
            #axs[0].plot(STH1_hist[0],STH1_hist[1] / np.sum(STH1_hist[1]),'-',label='1024',lw=curve_lw,color='r')
            axs[i].set_xlim(0,199)
            axs[i].set_title('Profile: {}'.format(c.addition_profile.pretty_name),size='30')
            axs[i].tick_params(axis='both', which='major', labelsize=24)

            c.clear_results()
  

    axs[0].set_ylabel('P(DM)',fontsize='30')
    axs[0].legend(loc='upper right',prop={'size':'25'})

    fig.text(0.5, 0.04, 'DM [pc cm$^{-3}$]', ha='center',fontsize=30)

    saveFig(filename, fig, bbox_inches='tight')

In [ ]:

# Make the two parts of the final figure in Appendix C of Khan, Williams, McQuinn 2022. 
# Compares effects of increasing the addition grid resolution
make_resolution_image_comparison(600, STH1_config, STH2_config, STH8_config, STH32_config)
make_resolution_DM_histogram_comparison([STH_configs, fire_configs])

# not used in paper but useful
make_resolution_DM_histogram_comparison([nfw_configs, precipitation_configs], filename="hist_resolution_nfw_precip")

In [ ]:
# Profiles for comparing 256 vs 512 base grid
STH_config_256 = Configuration(SphericalTophatProfile(), provider=provider, resolution=32)
STH_config_256.datestamp = date
STH_config_512 = Configuration(SphericalTophatProfile(), provider=provider, resolution=32, den_grid_size=512)
STH_config_512.datestamp = date

FIRE_config_256 = Configuration(FireProfile(), provider=provider, resolution=32)
FIRE_config_256.datestamp = date2
FIRE_config_512 = Configuration(FireProfile(), provider=provider, resolution=32, den_grid_size=512)
FIRE_config_512.datestamp = date2

NFW_config_256 = Configuration(NFWProfile(), provider=provider, resolution=32)
NFW_config_256.datestamp = date
NFW_config_512 = Configuration(NFWProfile(), provider=provider, resolution=32, den_grid_size=512)
NFW_config_512.datestamp = date

Precipitation_config_256 = Configuration(precipProfile, provider=provider, resolution=32)
Precipitation_config_256.datestamp = date2
Precipitation_config_512 = Configuration(precipProfile, provider=provider, resolution=32, den_grid_size=512)
Precipitation_config_512.datestamp = date2

STH_2_config_256 = Configuration(SphericalTophatProfile(rvir_factor=2), provider=provider, resolution=32)
STH_2_config_256.datestamp = date
STH_2_config_512 = Configuration(SphericalTophatProfile(rvir_factor=2), provider=provider, resolution=32, den_grid_size=512)
STH_2_config_512.datestamp = date


In [ ]:
# Histograms comparing 256 and 512 original grid for both raw and addition fields
def res_compare_256_512(config256, config512, name, resolution):

    fig_hist = plt.figure(figsize=(20,10))

    original_field_256 = provider.get_density_field(0, 256)
    original_field_512 = provider.get_density_field(0, 512)

    hist_256 = histArray(original_field_256,100,256,0,300)
    hist_512 = histArray(original_field_512,200,512,0,300)

    hist_256_rem = histArray(config256.get_final_field()[0,:,:],100,256*4*resolution,0,300)
    hist_512_rem = histArray(config512.get_final_field()[0,:,:],200,512*2*resolution,0,300)

    plt.plot(hist_256[0],hist_256[1],'-',label='256 Raw density field',lw=curve_lw,color='r')
    plt.plot(hist_512[0],hist_512[1],'-',label='512 Raw density field',lw=curve_lw,color='b')

    plt.plot(hist_256_rem[0],hist_256_rem[1],'--',label='256 Final density field',lw=curve_lw,color='r')        
    plt.plot(hist_512_rem[0],hist_512_rem[1],'--',label='512 Final density field',lw=curve_lw,color='b')

    plt.xlabel('DM [pc cm$^{-3}$]',fontsize='30')
    plt.ylabel('P(DM)',fontsize='30')
    plt.xlim(0,250)
    plt.legend(loc='upper right',prop={'size':'25'})
    plt.ylim(0,.021)

    # Something is broken with this for me TODO
    #f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    #g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.1e' % x))
    #plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(g))

    saveFig('hist_add_remove_halos_{}.pdf'.format(name), fig_hist, bbox_inches='tight')

res_compare_256_512(STH_config_256, STH_config_512, 'STH', 32)

#res_compare_256_512(FIRE_config_256, FIRE_config_512, 'FIRE', 8)


In [ ]:
# DM vs R Compare 512 and 256 res

xmin = 25
xmax = 2399

M_chosen = [10,22,34] 

# They are the same to 9 digits of precision; doesn't affect results
#print(np.mean(provider.get_density_field(0, 256)))
#print(np.mean(provider.get_density_field(0, 512)))

# Compare STH
ymax = [99,149,249]
series = [ 
    (STH_config_256.generate_DM_vs_radius_profile(load_from_files=True),STH_config_256.generate_profile_of_masks(load_from_files=True),'3D Tophat 256','r','-'), 
    (STH_config_512.generate_DM_vs_radius_profile(load_from_files=True),STH_config_512.generate_profile_of_masks(load_from_files=True),'3D Tophat 512','b','-') 
    ]
plots.make_DM_vs_Rad_profiles_plots(series, False, True, xmin, xmax, STH_config_256.resolution, STH_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'STH_256_vs_512')

# Compare Fire
ymax = [129,309,799] 
series = [ 
    (FIRE_config_256.generate_DM_vs_radius_profile(load_from_files=True),FIRE_config_256.generate_profile_of_masks(load_from_files=True),'Fire 256','r','-'), 
    (FIRE_config_512.generate_DM_vs_radius_profile(load_from_files=True),FIRE_config_512.generate_profile_of_masks(load_from_files=True),'Fire 512','b','-') 
    ]
plots.make_DM_vs_Rad_profiles_plots(series, False, True, xmin, xmax, FIRE_config_256.resolution, FIRE_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'FIRE_256_vs_512')

# Make the version we use in the paper
M_chosen = [22] 
ymax = [309] 
series = [ 
    (STH_config_256.generate_DM_vs_radius_profile(load_from_files=True),STH_config_256.generate_profile_of_masks(load_from_files=True),'3D Tophat 256','r','-'), 
    (STH_config_512.generate_DM_vs_radius_profile(load_from_files=True),STH_config_512.generate_profile_of_masks(load_from_files=True),'3D Tophat 512','b','-'),
    (NFW_config_256.generate_DM_vs_radius_profile(load_from_files=True),NFW_config_256.generate_profile_of_masks(load_from_files=True),'NFW 256','r','--'), 
    (NFW_config_512.generate_DM_vs_radius_profile(load_from_files=True),NFW_config_512.generate_profile_of_masks(load_from_files=True),'NFW 512','b','--') 
    ]
plots.make_DM_vs_Rad_profiles_plots(series, False, False, xmin, xmax, NFW_config_256.resolution, NFW_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'Paper_256_vs_512')

# Make a plot of the difference between 256 and 512
M_chosen = [10,22,34] 
mean_DM = np.mean(provider.get_density_field(0, 256)) # this gets subtracted off later we need to 'undo' that for the difference version of this
print(np.shape(STH_config_512.DM_vs_R1))
delta1 = (STH_config_512.DM_vs_R1 - STH_config_256.DM_vs_R1) + mean_DM
delta2 = (NFW_config_512.DM_vs_R1 - NFW_config_256.DM_vs_R1) + mean_DM

ymax = [49,49,49]
series = [ 
    (delta1,[],'3D Tophat $\Delta$','r','-'), 
    (delta2,[],'NFW $\Delta$','b','-') 
    ]
plots.make_DM_vs_Rad_profiles_plots(series, False, False, xmin, xmax, NFW_config_256.resolution, NFW_config_256.resolution*provider.halofieldresolution, M_chosen, STH_config_512.get_virial_radii(), provider, STH_config_512.get_halo_masses(), ymax, 'Delta_256_vs_512', y_label='$\Delta$ DM [pc cm$^{-3}$]')


In [ ]:
# Prints off means and standard deviations of the 256 and 512 base grid results for each profile (z=0 box)
def stats(profile, data_256, data_512):
    print('Mean of %s' % profile,'256 grid: ', np.round(data_256.mean(),2), '512 grid: ', np.round(data_512.mean(),2) )
    print('Standard Deviation of %s' % profile,'256 grid: ', np.round(data_256.std(),2), '512 grid: ', np.round(data_512.std(),2) )

stats('Tophat', STH_config_256.get_final_field(), STH_config_512.get_final_field())
stats('NFW', NFW_config_256.get_final_field(), NFW_config_512.get_final_field())
stats('Fire', FIRE_config_256.get_final_field(), FIRE_config_512.get_final_field())
stats('Precipitation', Precipitation_config_256.get_final_field(), Precipitation_config_512.get_final_field())
stats('Tophat 2', STH_2_config_256.get_final_field(), STH_2_config_512.get_final_field())
